# Watchlist example notebook

Here we will do a xmatch against a list of positions in the sky, given in the file ```watchlist.csv```

In [1]:
import pandas as pd
import psycopg2

# Get credentials to open a direct connection with the database

In [2]:
import json
# use the location of your credentials file here
credentials_file = "/home/fforster/Dropbox/Work/alercebroker/usecases/alercereaduser_v4.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

In [3]:
conn = psycopg2.connect(dbname=params["dbname"], user=params["user"], host=params["host"], password=params["password"])

# Get watchlist of objects

In [4]:
df = pd.read_csv("../example_data/watchlist.csv")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


# Prepare the query string for watchlist for object within 1 degress of given positions, and  happening within the last 10 days

In [5]:
objects = []
for _,row in df.iterrows():
    objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
objects_str = ",\n".join(objects)
objects_str

"('source_1', 160.18301441363647, 33.0164673528409),\n('source_2', 174.21524897555543, 44.83789535222221)"

Create query string

In [6]:
from astropy.time import Time
nt = Time.now()
last_mjd_discovery = nt.mjd - 10

In [7]:
search_radius = 1 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
)
SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd

FROM object o, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
""" % (last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [8]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(144, 8)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd
0,source_1,160.183014,33.016467,ZTF21aaqwdiv,159.371305,32.521336,0.843207,59301.299630
1,source_1,160.183014,33.016467,ZTF21aaqwdiu,159.756359,32.659909,0.505608,59301.299630
2,source_1,160.183014,33.016467,ZTF21aaqwdki,160.806774,32.632821,0.649563,59301.299630
3,source_1,160.183014,33.016467,ZTF21aaqypmo,159.991994,32.595622,0.450431,59303.259306
4,source_1,160.183014,33.016467,ZTF21aaqypmp,160.168464,32.870279,0.146698,59303.259306
...,...,...,...,...,...,...,...,...
139,source_1,160.183014,33.016467,ZTF21aasvhsy,159.297297,32.430306,0.948058,59310.311944
140,source_1,160.183014,33.016467,ZTF21aasvhta,160.157761,32.993761,0.031049,59310.311944
141,source_1,160.183014,33.016467,ZTF21aasvhso,161.249858,33.147698,0.903474,59310.311944
142,source_1,160.183014,33.016467,ZTF21aasvhsz,159.336525,33.232791,0.741189,59310.311944


# Query objects within 10 degress of given positions, first detected within the last 10 days, with SN probabilities > 0.4, and ranking=1 in the stamp classifier

In [9]:
search_radius = 10 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
),
sn (oid, classifier_name, class_name, probability, ranking) AS (
    SELECT
        o.oid, p.classifier_name, p.class_name, p.probability, p.ranking
    FROM
        probability p
    INNER JOIN 
        object o
    ON 
        o.oid=p.oid
    WHERE
        p.classifier_name='stamp_classifier'
        AND p.class_name IN ('SN', 'AGN')
        AND p.ranking=1
        AND p.probability > 0.4
        AND o.firstmjd > %s
)

SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd, sn.classifier_name, sn.class_name, sn.probability, sn.ranking

FROM object o INNER JOIN sn ON sn.oid=o.oid, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
ORDER BY
    c.source_id
""" % (last_mjd_discovery, last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [10]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches.head(100)

(313, 12)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd,classifier_name,class_name,probability,ranking
0,source_1,160.183014,33.016467,ZTF18acpmlui,166.640634,26.016864,8.971510,59308.339965,stamp_classifier,AGN,0.715071,1
1,source_1,160.183014,33.016467,ZTF18acxdfdo,171.094971,35.390679,9.326095,59305.288137,stamp_classifier,AGN,0.677221,1
2,source_1,160.183014,33.016467,ZTF20acsfwsy,149.798671,34.506218,8.756553,59306.236181,stamp_classifier,AGN,0.686381,1
3,source_1,160.183014,33.016467,ZTF21aaogfkk,168.123513,35.580565,7.040297,59304.173646,stamp_classifier,AGN,0.650135,1
4,source_1,160.183014,33.016467,ZTF21aakgowm,167.804548,32.316459,6.452625,59306.372836,stamp_classifier,AGN,0.674133,1
...,...,...,...,...,...,...,...,...,...,...,...,...
95,source_2,174.215249,44.837895,ZTF18aawxqhb,175.292711,35.805777,9.069154,59305.288137,stamp_classifier,AGN,0.425976,1
96,source_2,174.215249,44.837895,ZTF21aargssp,175.077453,35.803898,9.057726,59305.288137,stamp_classifier,AGN,0.697723,1
97,source_2,174.215249,44.837895,ZTF18acstzfc,175.990114,35.860073,9.078492,59305.288137,stamp_classifier,AGN,0.591342,1
98,source_2,174.215249,44.837895,ZTF18aawxqhu,176.032205,35.652406,9.288893,59305.288137,stamp_classifier,AGN,0.607130,1
